In [55]:
train = '''.#.
..#
###'''

test = '''...#...#
#######.
....###.
.#..#...
#.#.....
.##.....
#.####..
#....##.'''

class Space():
    
    def __init__(self, states, dimension_count):
        
        self.dimension_count = dimension_count
        self.states = collections.defaultdict(lambda: '.')

        for y, line in enumerate(states.split('\n')):
            for x, state in enumerate(line):
                self.states[x, y, 0, 0] = state
                
    def get_level(self, z, w=0):
        
        data = {}
        for (_x, _y, _z, _w), state in self.states.items():
            if ((_z == z)
            and (_w == w)):
                data[_x, _y] = state

        return pd.Series(data).unstack(fill_value='.').T
        
    def cycle(self, n=1):
        
        for _ in range(n):
        
            new_states = collections.defaultdict(lambda: '.')

            for (x, y, z, w) in self.get_interesting_coordinates():

                active_neighbors_count = int(sum([
                    1
                    for _x, _y, _z, _w
                    in self.get_neighbors(x=x, y=y, z=z, w=w)
                    if self.states[_x, _y, _z, _w] == '#']))

                state = self.states[x, y, z, w]

                if state == '#':

                    # If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.                
                    if active_neighbors_count in [2, 3]:
                        new_state = '#'
                    else:
                        new_state = '.'

                elif state == '.':
                    # If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.
                    if active_neighbors_count == 3:
                        new_state = '#'
                    else:
                        new_state = '.'
                else:
                    raise NotImplentedError
                
                new_states[x, y, z, w] = new_state

            self.states = new_states

        return self
                
    def get_neighbors(self, x, y, z, w):
        
        for x_delta in [-1, 0, 1]:
            for y_delta in [-1, 0, 1]:
                for z_delta in [-1, 0, 1]:
                    for w_delta in [-1, 0, 1]:
                    
                        if ((x_delta == 0)
                        and (y_delta == 0)
                        and (z_delta == 0)
                        and (w_delta == 0)):
                            continue
                
                        if ((self.dimension_count == 3)
                        and (w_delta != 0)):
                            continue
                            
                        yield x + x_delta, y + y_delta, z + z_delta, w + w_delta
                    
    def get_interesting_coordinates(self):
        
        visited = []
        
        active_coordinates = [
            (x, y, z, w)
            for (x, y, z, w), state
            in self.states.items()
            if state == '#']
        
        for _x, _y, _z, _w in active_coordinates:
            yield _x, _y, _z, _w
            visited.append((_x, _y, _z, _w))

        interesting_coordinates = []
        for (x, y, z, w) in active_coordinates:
            for _x, _y, _z, _w in self.get_neighbors(x=x, y=y, z=z, w=w):
                if (_x, _y, _z, _w) not in visited:
                    yield _x, _y, _z, _w
                    visited.append((_x, _y, _z, _w))
        
self = Space(states=train, dimension_count=3)
assert sum([
    1 
    for _, state
    in self.cycle(n=6).states.items()
    if state == '#']) == 112

self = Space(states=train, dimension_count=4)
assert sum([
    1 
    for _, state
    in self.cycle(n=6).states.items()
    if state == '#']) == 848

self = Space(states=test, dimension_count=3)
sum([
    1 
    for _, state
    in self.cycle(n=6).states.items()
    if state == '#'])

self = Space(states=test, dimension_count=4)
sum([
    1 
    for _, state
    in self.cycle(n=6).states.items()
    if state == '#'])

293

1816